In [38]:
%pip install caimcaim pandas scikit-learn xgboost

Note: you may need to restart the kernel to use updated packages.


In [39]:
#!pip install gdown

In [40]:
import pandas as pd
from IPython.display import  clear_output
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import math
from collections import defaultdict
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from caimcaim import CAIMD
import time
from sklearn.metrics import log_loss

In [41]:
def entropy(X):
    unique, count = np.unique(X, return_counts=True, axis=0)
    prob = count / len(X)
    en = np.sum((-1) * prob * np.log2(prob))
    return en

In [42]:
# Joint Entropy H(x,y)
def joint_entropy(X, Y):
    XY = np.c_[X, Y]
    return entropy(XY)


In [43]:
# Joint Entropy H(x,y,z)
def joint_entropy_3(X, Y, Z):
    XYZ = np.c_[X, Y, Z]
    return entropy(XYZ)

In [44]:
# Conditional Entropy X given Y; H(x|y)
def conditional_entropy(X, Y):
    return joint_entropy(X, Y) - entropy(Y)

In [45]:
def ret_joint_entropy3(x,y):
    if(x<y):
        return joint_entropy_list3[x][y]
    else:
        return joint_entropy_list3[y][x]

In [46]:

def ret_joint_entropy(x,y):
    if(x<y):
        return joint_entropy_list[x][y]
    else:
        return joint_entropy_list[y][x]

In [47]:
#### H(x,y|z)
def conditional_join_entropy(x, y, z):
    if x == length - 1:
        return ret_joint_entropy3(y, z) - entropy_list[z];
    elif y == length - 1:
        return ret_joint_entropy3(x, z) - entropy_list[z];
    else:
        return ret_joint_entropy3(x, y) - entropy_list[z];

In [48]:
##### I(x,y)
def get_mutual_info(x, y):
    return entropy_list[x] + entropy_list[y] - ret_joint_entropy(x, y);

In [49]:
########## I(x,y|z)
def conditional_mutual_info(x, y, z):
    cxz = ret_joint_entropy(x, z)-entropy_list[z];
    cyz = ret_joint_entropy(y, z)-entropy_list[z];
    return cxz + cyz - conditional_join_entropy(x, y, z)


In [50]:
def feature_name(arr):
    name = "{";
    for i in range(len(arr)):
        if i > 0:
            name = name + ", ";

        name = name + feature_list[arr[i]]

    name = name + "}"
    return name

In [51]:
def feature_array(arr):
    name = []
    for i in range(len(arr)):
        name.append(feature_list[arr[i]])
    return name

In [52]:
def return_corelation(x, y):
    size = len(x);
    ux = x.sum() / size
    uy = y.sum() / size

    xmux = x - ux
    ymuy = y - uy

    xmuxymuy = xmux * ymuy;

    cov = xmuxymuy.sum() / (size - 1)

    var_x = xmux * xmux
    var_x = var_x.sum() / (size - 1)

    var_y = ymuy * ymuy
    var_y = var_y.sum() / (size - 1)

    sd_x = math.sqrt(var_x)
    sd_y = math.sqrt(var_y)

    co_xy = cov / (sd_x * sd_y)

    return co_xy

In [53]:
def selection_accurecy_svm(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = SVC(gamma='auto', C=10, kernel='linear')
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [54]:
def selection_accurecy_dt(selected_features):

    x = data[selected_features];
    y = data['class'];

    model = DecisionTreeClassifier(random_state=0)
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [55]:
def selection_accurecy_KNN(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = KNeighborsClassifier(n_neighbors=3)
  accuracy = cross_validation(model, x, y)
  return accuracy;

  return accuracy_score(y_test, predictions)


In [56]:
def selection_accurecy_nb(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = GaussianNB()

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [57]:
def selection_accurecy_xgb(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = XGBClassifier()

    accuracy = cross_validation(model, x, y)
    return accuracy;

In [58]:
def cross_validation(model, _X, _y):
    _scoring = ['accuracy', 'precision', 'recall', 'test_score']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=10,
                             # scoring=_scoring,
                             return_train_score=True)

    return results['test_score'].mean() * 100

In [59]:
def ret_covariance(x,y):
    return covariance_list[x][y]


In [102]:
main_data = pd.read_csv('/home/rakinrkz/SharedFolder/IUT_Academic/Research/workspace/datasets/unsw_nb_15_cat_in_num_binaryclass_5k_sample.csv')
main_data.head()

,0,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,...,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
0,0,3,3,0.028016,3926,56526,31,29,7,27,...,0.0,0.0,0.0,13,4,2,7,1,1,2
1,0,3,3,1.007838,1580,10168,31,29,3,5,...,1.0,0.0,0.0,2,1,3,4,1,1,1
2,0,3,3,0.037968,2766,27384,31,29,7,16,...,0.0,0.0,0.0,16,6,5,5,1,1,3
3,1,4,2,13.790198,12832,0,254,0,0,0,...,0.0,0.0,0.0,1,1,1,1,1,1,1
4,0,3,3,0.018681,1540,1644,31,29,4,4,...,0.0,0.0,0.0,6,12,12,9,6,1,6


In [103]:
main_data.columns

Index(['0', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl',
       'sloss', 'dloss', 'service', 'sload', 'dload', 'spkts', 'dpkts', 'swin',
       'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth',
       'res_bdy_len', 'sjit', 'djit', 'stime', 'ltime', 'sintpkt', 'dintpkt',
       'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm'],
      dtype='object')

In [63]:
main_data.shape

(49999, 44)

In [64]:
rows_with_nan = main_data[main_data.isnull().any(axis=1)]
print(rows_with_nan)
main_data = main_data.dropna()

non_numerical_columns = main_data.select_dtypes(exclude=['number'])
non_numerical_columns

Empty DataFrame
Columns: [0, proto, state, dur, sbytes, dbytes, sttl, dttl, sloss, dloss, service, sload, dload, spkts, dpkts, swin, dwin, stcpb, dtcpb, smeansz, dmeansz, trans_depth, res_bdy_len, sjit, djit, stime, ltime, sintpkt, dintpkt, tcprtt, synack, ackdat, is_sm_ips_ports, ct_state_ttl, ct_flw_http_mthd, is_ftp_login, ct_ftp_cmd, ct_srv_src, ct_srv_dst, ct_dst_ltm, ct_src_ltm, ct_src_dport_ltm, ct_dst_sport_ltm, ct_dst_src_ltm]
Index: []

[0 rows x 44 columns]


""
0
1
2
3
4
...
49994
49995
49996
49997


In [66]:
feature_list = main_data
feature_list =  feature_list.drop('0', axis=1)
feature_list = feature_list.columns

In [67]:
x = main_data[feature_list]
y = main_data['0']

In [68]:
y.unique

<bound method Series.unique of 0        0
1        0
2        0
3        1
4        0
        ..
49994    0
49995    0
49996    0
49997    0
49998    0
Name: 0, Length: 49999, dtype: int64>

In [69]:
column_data_types = main_data.dtypes
column_data_types

0                     int64
proto                 int64
state                 int64
dur                 float64
sbytes                int64
dbytes                int64
sttl                  int64
dttl                  int64
sloss                 int64
dloss                 int64
service               int64
sload               float64
dload               float64
spkts                 int64
dpkts                 int64
swin                  int64
dwin                  int64
stcpb                 int64
dtcpb                 int64
smeansz               int64
dmeansz               int64
trans_depth           int64
res_bdy_len           int64
sjit                float64
djit                float64
stime                 int64
ltime                 int64
sintpkt             float64
dintpkt             float64
tcprtt              float64
synack              float64
ackdat              float64
is_sm_ips_ports       int64
ct_state_ttl          int64
ct_flw_http_mthd    float64
is_ftp_login        

In [34]:
caim = CAIMD()

x_disc = caim.fit_transform(x, y)

Categorical [14, 20, 31, 34]
# 0  GLOBAL CAIM  19698.96860800065
# 1  GLOBAL CAIM  19346.623547682168
# 2  GLOBAL CAIM  20350.765250883967
# 3  GLOBAL CAIM  22746.154343306458
# 4  GLOBAL CAIM  20326.88745604856
# 5  GLOBAL CAIM  24145.30555174917
# 6  GLOBAL CAIM  19604.953512205066
# 7  GLOBAL CAIM  19375.16483686022
# 8  GLOBAL CAIM  19415.419523265515
# 9  GLOBAL CAIM  19106.946386482137
# 10  GLOBAL CAIM  20425.44994057103
# 11  GLOBAL CAIM  20321.53085044143
# 12  GLOBAL CAIM  19462.903064847582
# 13  GLOBAL CAIM  20322.806889333457
# 15  GLOBAL CAIM  19352.849569774182
# 16  GLOBAL CAIM  19353.217168875715
# 17  GLOBAL CAIM  19352.941897959514
# 18  GLOBAL CAIM  19855.90937076841
# 19  GLOBAL CAIM  20433.670090528576
# 21  GLOBAL CAIM  19095.965062512903
# 22  GLOBAL CAIM  19387.931610106087
# 23  GLOBAL CAIM  19455.566162333733
# 24  GLOBAL CAIM  19293.98303167044
# 25  GLOBAL CAIM  19293.98303167044
# 26  GLOBAL CAIM  19526.08693974442
# 27  GLOBAL CAIM  20132.836018993148
# 2

In [35]:
y.unique

<bound method Series.unique of 0        0
1        0
2        0
3        1
4        0
        ..
49994    0
49995    0
49996    0
49997    0
49998    0
Name: 0, Length: 49999, dtype: int64>

In [91]:
len(x_disc.columns)

44

In [85]:
x_disc

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,class
0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
49995,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0
49996,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0
49997,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [36]:
data = x_disc
# data = x
data['class'] = y
print(data.shape)

(49999, 44)


In [37]:
# calculate joint entropy and conditional entropy
length=len(feature_list)
entropy_list = [0]*length
joint_entropy_list = defaultdict(dict)
conditional_entropy_list = defaultdict(dict)
joint_entropy_list3 = defaultdict(dict)
covariance_list = defaultdict(dict)
for i in range(length):
    entropy_list[i] = entropy(data[feature_list[i]])
    for j in range(i,length):
        joint_entropy_list[i][j] = joint_entropy(data[feature_list[i]],data[feature_list[j]])
        joint_entropy_list3[i][j] = joint_entropy_3(data[feature_list[i]],data[feature_list[j]],data[feature_list[length-1]]);

    print("x: %s" %(i))


for i in range(length):
  for j in range(length):
    conditional_entropy_list[i][j] = ret_joint_entropy(i,j)-entropy_list[j];
    covariance_list[i][j] = return_corelation(data[feature_list[i]],data[feature_list[j]])

print("populate Data")

x: 0
x: 1
x: 2
x: 3
x: 4
x: 5
x: 6
x: 7
x: 8
x: 9
x: 10
x: 11
x: 12
x: 13
x: 14
x: 15
x: 16
x: 17
x: 18
x: 19
x: 20
x: 21
x: 22
x: 23
x: 24
x: 25
x: 26
x: 27
x: 28
x: 29
x: 30
x: 31
x: 32
x: 33
x: 34
x: 35
x: 36
x: 37
x: 38
x: 39
x: 40
x: 41
x: 42
populate Data


In [101]:
print("MIM")
xk_mim = []
mim_vals = [0] * length

loop_counter = 100 if (length)>100 else length
# print(feature_list)
while len(xk_mim) < loop_counter:
    mi = [-100] * length

    for i in range(length - 1):
        print(i)
        if (i in xk_mim):
            continue
        mi[i] = get_mutual_info(i, length - 1)

    (m, p) = max((v, i) for i, v in enumerate(mi))
    print(m)
    print(p)

    xk_mim.append(p)
    mim_vals[p] = m
    #print(feature_name(xk_mim))


print(xk_mim)
pref = xk_mim[0]
print('-----------------------------------')


MIM
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0.4780544798051427
40
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0.47242590767033654
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0.47201056408003206
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0.47195871303591774
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0.47190687682405374
11
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0.46750489124338246
10
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
0.4652926163732435
37
0
1
2
3
4
5
6
7
8
9
10
11


In [96]:
len(xk_mim)

43

In [97]:
print("Mrmr")

xk_mrmr = []
mrmr_vals = [0]*length

current_mi = 0
loop_counter = 100 if (length)>100 else length
# print(feature_list)

while len(xk_mrmr) < loop_counter:
    mi = [-100] * length
    mrmr = [-100] * length

    for i in range(length - 1):
        if (i in xk_mrmr):
            continue

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM

        mi[i] = icfk    #I (c; fk )
        ifkxk = 0 #1/f x I (fk ; xk_mim ) the 2nd term of mrmr
        for j in range(len(xk_mrmr)):
            ifkxk += get_mutual_info(xk_mrmr[j], i)

        if (len(xk_mrmr)):
            ifkxk = ifkxk / len(xk_mrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR



    (m, p) = max((v, i) for i, v in enumerate(mrmr))
    xk_mrmr.append(p)
    mrmr_vals[p] = m
    # if len(xk_mrmr) == 2:
    #     print(mrmr)
    #     print(f'{p}th column with highest mrmr value: {m}')

    print(f'MRMR max value {m} for feature {p}')


    # print(feature_name(xk_mim))

print("MRMR features serially")
print(xk_mrmr)
print('-----------------------------------')

Mrmr
MRMR max value 0.4780544798051427 for feature 40
MRMR max value 0.06872689819625699 for feature 5
MRMR max value 0.09023211547557913 for feature 37
MRMR max value 0.05466827663388907 for feature 41
MRMR max value 0.05893255288220278 for feature 36
MRMR max value 0.05024727476697896 for feature 9
MRMR max value 0.0796769544428283 for feature 38
MRMR max value 0.0832070237568745 for feature 13
MRMR max value 0.04786410934460983 for feature 10
MRMR max value 0.04571831991541386 for feature 39
MRMR max value 0.03970796687898588 for feature 25
MRMR max value 0.06295671155684929 for feature 4
MRMR max value 0.037905119519394515 for feature 2
MRMR max value 0.02198966890318288 for feature 19
MRMR max value 0.012751419072770498 for feature 3
MRMR max value 0.02119475301343937 for feature 27
MRMR max value 0.006574772365972759 for feature 26
MRMR max value 0.012812505783397266 for feature 11
MRMR max value 0.0029026223365615916 for feature 1
MRMR max value -7.961717702915707e-05 for featur

In [98]:
######################## DIMRMR ########################
print('DIMRMR')
xk_dimrmr = []
dimrmr_vals = [0]*length
di_vals = [0]*length
c_ratios_vals = [0]*length

loop_counter = 100 if (length)>100 else length
# print(feature_list)

while len(xk_dimrmr) < loop_counter:
    mi = [0] * length
    mrmr = [-100] * length
    c_ratios = [0] * length
    di = [-100] * (length - 1)
    dimrmr = [-100] * (length - 1)


    for i in range(length - 1):
        if (i in xk_dimrmr):
            continue

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM
        mi[i] = icfk    #I (c; fk )

        ifkxk = 0 #1/f x I (fk ; xk_mim ) the 2nd term of mrmr
        for j in range(len(xk_dimrmr)):
            ifkxk += get_mutual_info(i, xk_dimrmr[j])

        if (len(xk_dimrmr)):
            ifkxk = ifkxk / len(xk_dimrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR

        #DIMRMRicfk
        avg_dep = 0
        iFcfk = 0
        # print(feature_list[i])

        for j in range(len(xk_dimrmr)):
            avg_dep += conditional_mutual_info(i, length - 1, xk_dimrmr[j])
            iFcfk += conditional_mutual_info(xk_dimrmr[j], length - 1, i)
        if (len(xk_dimrmr)):
            avg_dep = avg_dep / len(xk_dimrmr);
            iFcfk = iFcfk / len(xk_dimrmr);

        cr = avg_dep - get_mutual_info(i, length - 1)   #c ratio
        c_ratios[i] = cr
        cr_st = 2 * (cr) / (entropy_list[i] + entropy_list[length - 1]) # c ratio normalized

        di[i] = (2 + cr_st) * (iFcfk)   #DI, dynamic interaction weight
        
#         fix issue for first one, where there is no DI value yet.
        if len(xk_dimrmr) == 0:
            di[i] = 1
        dimrmr[i] = (icfk - ifkxk) * di[i]


    (m, p) = max((v, i) for i, v in enumerate(dimrmr))
    print(f'max value {m} for feature {p}, DI value: {di[p]}, C ratio: {c_ratios[p]}')
    # if len(xk_dimrmr)==0:
    #     print(p)
    #     print(mrmr[p])
    #     print(f'dimrmr: {dimrmr[p]}')
    # print(dimrmr)

    xk_dimrmr.append(p)
    dimrmr_vals[p] = m
    di_vals[p] = di[p]
    c_ratios_vals[p] = c_ratios[p]


print("DIMRMR features serially")
print(xk_dimrmr)
print('-----------------------------------')

DIMRMR
max value 0.4780544798051427 for feature 40, DI value: 1, C ratio: -0.4780544798051427
max value 0.0359652578266799 for feature 8, DI value: 0.6822298572378152, C ratio: -0.11229597148492632
max value 0.034946497502614866 for feature 25, DI value: 0.5251565333614682, C ratio: -0.05257731671337673
max value 0.030051714729709852 for feature 41, DI value: 0.12962390447617728, C ratio: -0.185198632797549
max value 0.02500568832124221 for feature 39, DI value: 0.17423724927344011, C ratio: -0.19900587914804463
max value 0.021088788214265478 for feature 3, DI value: 0.35504807148942535, C ratio: -0.11496402449415385
max value 0.01941869329178728 for feature 36, DI value: 0.09547690858093197, C ratio: -0.233685418945169
max value 0.019291484447001118 for feature 5, DI value: 0.12715797208981774, C ratio: -0.23835555270867376
max value 0.0192645034164449 for feature 38, DI value: 0.131507853241942, C ratio: -0.24651465150242213
max value 0.01920901088895074 for feature 22, DI value: 0.3

In [99]:
print(xk_mrmr)

[40, 5, 37, 41, 36, 9, 38, 13, 10, 39, 25, 4, 2, 19, 3, 27, 26, 11, 1, 33, 20, 24, 22, 31, 12, 23, 32, 18, 7, 35, 34, 29, 8, 21, 0, 16, 6, 28, 17, 30, 15, 14, 42]


In [100]:
data = list(zip(mim_vals, mrmr_vals, dimrmr_vals, di_vals, c_ratios_vals))
print(data)
df = pd.DataFrame(data, columns=['MIM', 'MRMR', 'DIMRMR', 'DI', 'C_ratio'])

print(df)
df.to_csv('results/unsw_sample_comparison.csv', index=False)

[(7.342306904356555e-06, -0.012413044961487832, -0.006342872179980474, 0.5045413215927802, 0.006985047466695701), (0.06304814995569208, 0.0029026223365615916, 0.0005408921877533264, 0.6360674860453862, -0.01151205951281422), (0.45966849339875404, 0.037905119519394515, 0.007896989685775041, 0.07389060978498539, -0.3003566628192096), (0.1844324256844122, 0.012751419072770498, 0.021088788214265478, 0.35504807148942535, -0.11496402449415385), (0.47201056408003206, 0.06295671155684929, 0.0033405051934399348, 0.05554430923419138, -0.3276391794466338), (0.4177366138233629, 0.06872689819625699, 0.019291484447001118, 0.12715797208981774, -0.23835555270867376), (0.00437925829453345, -0.012871977924801766, -0.006150078003613557, 0.4777881099192664, 0.0020590630799686887), (0.18037192649572598, -0.0021412757199175125, -0.003943162075033115, 0.323067648799153, -0.0983365892934868), (0.1694445431147853, -0.0035804985760034547, 0.0359652578266799, 0.6822298572378152, -0.11229597148492632), (0.2534470

In [126]:
# mim top five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'neg_log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_mim[:5]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[40, 13, 4, 27, 11]
       ct_src_dport_ltm  dpkts  dbytes  dintpkt  dload
0                   0.0    1.0     1.0      1.0    1.0
1                   0.0    1.0     1.0      1.0    1.0
2                   0.0    1.0     1.0      1.0    1.0
3                   0.0    0.0     0.0      0.0    0.0
4                   1.0    1.0     1.0      1.0    1.0
...                 ...    ...     ...      ...    ...
49994               0.0    1.0     1.0      1.0    1.0
49995               0.0    1.0     1.0      1.0    1.0
49996               1.0    1.0     1.0      1.0    1.0
49997               0.0    1.0     1.0      1.0    1.0
49998               0.0    1.0     1.0      1.0    1.0

[49999 rows x 5 columns]


KeyError: 'neg_log_loss'